# Imports

In [4]:
import pandas as pd
import glob
import numpy as np
import mysql.connector as mysql
import warnings
from mysql.connector import Error

# Ignore Warnings
from pandas.errors import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

# ELT

## Read all the files

In [5]:
ht_path = "High Tension/"
lt_path = "Low Tension/"
ht_files = glob.glob(ht_path + "*.csv")
lt_files = glob.glob(lt_path + "*.csv")
ht_files.sort()
lt_files.sort()

## Files corresponding and its variables
*   0 --> 2018-19
*   1 --> 2019-20
*   2 --> 2020-21
*   3 --> 2021-22
*   4 --> 2022-23







In [6]:
ht = []
lt = []
for i in range(len(ht_files)):
  ht.append(pd.read_csv(ht_files[i]))
  lt.append(pd.read_csv(lt_files[i]))

## Get Index of NULL Values in the data

In [7]:
ht_index = []
lt_index = []
for i in range(len(ht)):
  ht_index.append(np.where(ht[i]['Sl. No.'].isnull() == True)[0])
  lt_index.append(np.where(lt[i]['Sl. No.'].isnull() == True)[0])

## Change adding Bengaluru to end of cities

In [8]:
for i in range(len(ht)):
    ht[i]['Divisions'] = ht[i]['Divisions'] + ", Bengaluru"
    lt[i]['Divisions'] = lt[i]['Divisions'] + ", Bengaluru"

## Changing Sl No so that it division corresponds with its area

In [9]:
for x in range(len(ht)):
    # For ht data
    ht_prev = 0
    ht_mapper = 1
    for i in ht_index[x]:
        ht[x]['Sl. No.'].iloc[ht_prev:i] = ht_mapper
        ht_mapper += 1
        ht_prev = i + 1
        
    # For lt data
    lt_prev = 0
    lt_mapper = 1
    for i in ht_index[x]:
        lt[x]['Sl. No.'].iloc[lt_prev:i] = lt_mapper
        lt_mapper += 1
        lt_prev = i + 1

## Creating new DataFrames to store Null values which are wards in Bengaluru

In [10]:
ht_area_data = []
lt_area_data = []
for i in range(len(ht)):
    ht_area_data.append(pd.DataFrame(data=ht[i].iloc[ht_index[i]]))
    lt_area_data.append(pd.DataFrame(data=lt[i].iloc[lt_index[i]]))

,Sl. No.,Divisions,Overhead(OH) Added for 2019-20,OH Dismantled in 2019-20,OH Cables Total,Underground(UG) Added for 2019-20,UG Dismantled in 2019-20,UG Total,Ariel Bundled(AB) Cables Added for 2019-20,AB Cables Dismantled in 2019-20,AB Cables Total,Total Cable Length (km)
4,1.0,"East Circle, Bengaluru",1.040,0,775.9300,96.150,0,1327.565,47.04,0.0,114.970,2218.4650
8,1.0,"South Circle, Bengaluru",10.180,0,2390.0900,40.560,0,1633.760,1.57,0.0,119.360,4143.2100
12,1.0,"West Circle, Bengaluru",12.654,0,1354.6520,41.772,0,725.147,0.67,0.0,299.817,2379.6160
16,1.0,"North Circle, Bengaluru",10.780,0,1323.1300,39.290,0,892.463,0.10,0.0,94.940,2310.5330
19,1.0,"Blore Rural Circle, Bengaluru",447.480,0,11722.9520,19.250,0,569.248,0.63,0.0,3.000,12295.2000
24,1.0,"Ramnagar Circle, Bengaluru",897.587,0,9905.3383,11.604,0,245.482,7.11,0.0,146.950,10297.7703
29,1.0,"Kolar Circle, Bengaluru",775.110,0,16430.5500,0.000,0,174.170,0.00,0.0,3.270,16607.9900
34,1.0,"Tumkur Circle, Bengaluru",2931.879,0,29810.0260,0.000,0,0.000,0.00,0.0,0.000,29810.0260
39,1.0,"Davanagere Circle, Bengaluru",1575.720,0,27742.2400,0.000,0,9.055,0.00,0.0,14.460,27765.7550
40,1.0,"BESCOM, Bengaluru",6662.430,0,101454.9083,248.626,0,5576.890,3.22,0.0,796.767,107828.5653


## Making SL. No. of wards and their respective areas same

In [11]:
for i in ht_area_data[0].index:
    ind = 1
    ht_area_data[0].at[i, 'Sl. No.'] = ind

## Dropping the null values from the database

In [12]:
for i in range(len(ht)):
    ht[i].dropna(inplace=True)
    lt[i].dropna(inplace=True)

# Create Database

In [16]:
try:
    conn = mysql.connect(host='localhost', user='root', password='root', database='bescom')
    if conn.is_connected():
        cursor = conn.cursor()
        print("Connection Successfull !!")
except Error as e:
        print(e)

Connection Successfull !!
